# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [ ]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [ ]:
# load messages dataset
messages = pd.read_csv('workspace/data/disaster_messages.csv')
messages.head()

In [ ]:
# load categories dataset
categories = pd.read_csv('workspace/data/disaster_categories.csv')
categories.head()

In [ ]:
print(messages.shape)
print(categories.shape)


### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [ ]:
# merge datasets
df = pd.merge(messages, categories, on='id')

df.head()


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [ ]:
# take the categories column and split the values that are seperated by ; into a series
# create a dataframe of the 36 individual category columns
# categories = 
# categories.head()
categories_series = df['categories'].str.split(";")

categories_list = categories_series.to_list()
categories_dataframe = pd.DataFrame(categories_list)

categories_dataframe.head()



# (26248, 36)

In [ ]:
# output first row
categories_dataframe.iloc[0]

In [ ]:
# output first field
categories_dataframe.iloc[0][0]

In [ ]:
# select the first row of the categories dataframe
# select the first row of the categories dataframe
row = categories_dataframe.iloc[0]

# remove all -1
row = row.replace({'-1': ''}, regex=True)

# remove all -0
row = row.replace({'-0': ''}, regex=True)

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row
print(category_colnames)

In [ ]:
# rename the columns of `categories`
categories_dataframe.columns = category_colnames
categories_dataframe.head()

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [ ]:
for column in categories_dataframe:
   
    # remove all the column words
    categories_dataframe[column] = categories_dataframe[column].replace({column: ''}, regex=True)    

    # remove all the -
    categories_dataframe[column] = categories_dataframe[column].replace({'-': ''}, regex=True)      
    
    # convert column from string to numeric    
#     categories_dataframe.astype({column: 'int32'}).dtypes
    categories_dataframe[column] = categories_dataframe[column].astype(int)

print(categories_dataframe.dtypes)
categories_dataframe.head()

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [ ]:
df['categories']

In [ ]:
# drop the original categories column from `df`
df = df.drop(columns=['categories'])

df.head()

In [ ]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories_dataframe], axis=1, sort=False)

df.head()

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
# check number of duplicates
print('Number of rows_______:',  df.shape[0])
print('Number of unique rows:', len(df.drop_duplicates()))
print('Number of duplicates_:', df.shape[0] - len(df.drop_duplicates()))


In [ ]:
# drop duplicates
df = df.drop_duplicates()

In [ ]:
# check number of duplicates
print('Number of rows_______:',  df.shape[0])
print('Number of unique rows:', len(df.drop_duplicates()))
print('Number of duplicates_:', df.shape[0] - len(df.drop_duplicates()))

In [ ]:
df.shape

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [ ]:
engine = create_engine('sqlite:///DisasterResponse2.db')
try:
    df.to_sql('MessageClass', engine, index=False, if_exists='replace')
    print('Table created')
except:
  print('Error creating table')

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.